In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
""""""!pip uninstall -y langchain langchain-core langchain-community langchain-huggingface

!pip install langchain==0.1.14 \
            langchain-core==0.1.33 \
            langchain-community==0.0.23 \
            langchain-huggingface==0.0.5

!pip install -U pydantic

!pip install langchain_community

SyntaxError: invalid syntax (3654565852.py, line 1)

In [3]:
!pip uninstall -y transformers sentence-transformers peft
!pip install transformers==4.41.2 sentence-transformers==2.6.1 peft==0.10.0
# (Optional, only if you see a warning about HuggingFaceEmbeddings)
!pip install -U langchain-huggingface
!pip install langchain_community
!pip install chromadb



Found existing installation: transformers 4.52.4
Uninstalling transformers-4.52.4:
  Successfully uninstalled transformers-4.52.4
Found existing installation: sentence-transformers 4.1.0
Uninstalling sentence-transformers-4.1.0:
  Successfully uninstalled sentence-transformers-4.1.0
Found existing installation: peft 0.15.2
Uninstalling peft-0.15.2:
  Successfully uninstalled peft-0.15.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 64.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 90.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("hglm")


In [5]:
from huggingface_hub import login
login(secret_value_0)

In [4]:
!pip install langchain-huggingface


  Using cached langchain_huggingface-0.3.1-py3-none-any.whl.metadata (996 bytes)
Using cached langchain_huggingface-0.3.1-py3-none-any.whl (27 kB)


In [10]:
import os
import time
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# Step 1: Load and split PDFs
def load_pdf_folder(folder_path):
    start_time = time.time()
    documents = []
    for file in os.listdir(folder_path):
        if file.endswith(".pdf"):
            loader = PyPDFLoader(os.path.join(folder_path, file))
            documents.extend(loader.load())
    print(f"📄 Loaded {len(documents)} documents in {time.time() - start_time:.2f} seconds")
    return documents

# Step 2: Split into chunks
def split_docs(documents):
    start_time = time.time()
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = splitter.split_documents(documents)
    print(f"✂️  Split into {len(chunks)} chunks in {time.time() - start_time:.2f} seconds")
    return chunks

# Step 3: Embed documents
def create_vectorstore(docs):
    start_time = time.time()
    embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    db = Chroma.from_documents(docs, embedding)
    print(f"🔍 Created vectorstore in {time.time() - start_time:.2f} seconds")
    return db

# Step 4: Load Mistral 7B (recommended replacement)
def load_mistral_model(model_id="mistralai/Mistral-7B-Instruct-v0.2"):
    start_time = time.time()
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype="auto"
    )
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1
    )
    llm = HuggingFacePipeline(pipeline=pipe)
    print(f"🤖 Loaded Mistral 7B in {time.time() - start_time:.2f} seconds")
    return llm

# Step 5: Answer a question using RAG
def ask_question(llm, db, question):
    start_time = time.time()
    relevant_docs = db.similarity_search(question, k=2)
    
    context_parts = []
    for doc in relevant_docs:
        clean_text = doc.page_content.strip()
        if len(clean_text) > 500:
            clean_text = clean_text[:500] + "..."
        context_parts.append(clean_text)
    
    context = "\n\n".join(context_parts)
    
    prompt = f"""You are a helpful assistant. Based on the following context, answer the question concisely.

Context:
{context}

Question: {question}
Answer:"""
    
    response = llm.invoke(prompt)

    response_text = str(response)

    # Extract only the answer part, without the context or prompt
    answer_start = response_text.find("Answer:")
    if answer_start != -1:
        answer = response_text[answer_start + len("Answer:"):].strip()
    else:
        answer = response_text.strip()

    # Clean out any remaining prompt artifacts just in case
    for artifact in ["Context:", "Question:", "Answer the question using the context below."]:
        answer = answer.replace(artifact, "")
    answer = answer.strip()
    print(f"⏱️  Generated answer in {time.time() - start_time:.2f} seconds")
    
    return answer



# Step 6: Chatbot loop
def chatbot(llm, db):
    print("\n" + "="*80)
    print("🤖 RAG CHATBOT - Ask questions about your documents!")
    print("💡 Type 'quit' or 'exit' to end the conversation")
    print("="*80)

    question_count = 0

    while True:
        print("\n" + "-"*50)
        user_input = input("❓ Your question: ").strip()

        if user_input.lower() in ['quit', 'exit', 'q', 'bye']:
            print("\n👋 Thanks for using the RAG Chatbot! Goodbye!")
            break

        if not user_input:
            print("⚠️  Please enter a question.")
            continue

        question_count += 1
        print(f"\n🔍 Processing question #{question_count}...")

        try:
            answer = ask_question(llm, db, user_input)
            print(f"\n💡 Answer #{question_count}:")
            print("-" * 40)
            print(answer)
            print("-" * 40)
        except Exception as e:
            print(f"❌ Error generating answer: {str(e)}")

# === MAIN ===
pdf_folder_path = "/kaggle/input/documents/"  # Update if needed

print("🚀 Starting RAG Pipeline...")
total_start_time = time.time()

print("🔍 Loading PDFs...")
docs = load_pdf_folder(pdf_folder_path)
chunks = split_docs(docs)

print("📚 Creating vectorstore...")
db = create_vectorstore(chunks)

print("🚀 Loading Mistral model...")
llm = load_mistral_model()

print(f"✅ Setup completed in {time.time() - total_start_time:.2f} seconds")

chatbot(llm, db)

print(f"\n🎉 Total session time: {time.time() - total_start_time:.2f} seconds")


🚀 Starting RAG Pipeline...
🔍 Loading PDFs...
📄 Loaded 153 documents in 9.98 seconds
✂️  Split into 2103 chunks in 0.05 seconds
📚 Creating vectorstore...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


🔍 Created vectorstore in 7.37 seconds
🚀 Loading Mistral model...


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

...

❓ Your question:  Which factors affect brand competitiveness?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



🔍 Processing question #3...
⏱️  Generated answer in 20.12 seconds

💡 Answer #3:
----------------------------------------
Brand competitiveness is influenced by a combination of factors including a brand's uniqueness based on internal features, external image, and regional characteristics, as well as its market share and value creation capability.
----------------------------------------

--------------------------------------------------


❓ Your question:  quit



👋 Thanks for using the RAG Chatbot! Goodbye!

🎉 Total session time: 314.87 seconds


In [13]:
import os
import time
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# Step 1: Load and split PDFs
def load_pdf_folder(folder_path):
    start_time = time.time()
    documents = []
    for file in os.listdir(folder_path):
        if file.endswith(".pdf"):
            loader = PyPDFLoader(os.path.join(folder_path, file))
            documents.extend(loader.load())
    print(f"📄 Loaded {len(documents)} documents in {time.time() - start_time:.2f} seconds")
    return documents

# Step 2: Split into chunks
def split_docs(documents):
    start_time = time.time()
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = splitter.split_documents(documents)
    print(f"✂️  Split into {len(chunks)} chunks in {time.time() - start_time:.2f} seconds")
    return chunks

# Step 3: Embed documents
def create_vectorstore(docs):
    start_time = time.time()
    embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    db = Chroma.from_documents(docs, embedding)
    print(f"🔍 Created vectorstore in {time.time() - start_time:.2f} seconds")
    return db

# Step 4: Load Falcon 7B Instruct
def load_falcon_model(model_id="tiiuae/falcon-7b-instruct"):
    start_time = time.time()
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype="auto",
        trust_remote_code=True
    )
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1
    )
    llm = HuggingFacePipeline(pipeline=pipe)
    print(f"🤖 Loaded Falcon 7B Instruct in {time.time() - start_time:.2f} seconds")
    return llm

# Step 5: Ask question function (clean answer only)
def ask_question(llm, db, question):
    start_time = time.time()
    relevant_docs = db.similarity_search(question, k=2)
    
    context_parts = []
    for doc in relevant_docs:
        clean_text = doc.page_content.strip()
        if len(clean_text) > 500:
            clean_text = clean_text[:500] + "..."
        context_parts.append(clean_text)
    
    context = "\n\n".join(context_parts)
    
    prompt = f"""You are a helpful assistant. Based on the following context, answer the question concisely.

Context:
{context}

Question: {question}
Answer:"""
    
    response = llm.invoke(prompt)
    response_text = str(response)
    
    # Extract answer after "Answer:" label if present
    answer_start = response_text.find("Answer:")
    if answer_start != -1:
        answer = response_text[answer_start + len("Answer:"):].strip()
    else:
        answer = response_text.strip()
    
    # Clean any residual artifacts
    for artifact in ["Context:", "Question:", "Answer the question using the context below."]:
        answer = answer.replace(artifact, "")
    answer = answer.strip()
    
    print(f"⏱️ Generated answer in {time.time() - start_time:.2f} seconds")
    return answer

# Step 6: Chatbot loop for Falcon + vectorstore
def chatbot(llm, db):
    print("\n" + "="*80)
    print("🤖 Falcon 7B RAG Chatbot")
    print("💡 Type 'quit' or 'exit' to end the session")
    print("="*80)

    question_count = 0

    while True:
        user_input = input("\n❓ Your question: ").strip()

        if user_input.lower() in ['quit', 'exit', 'q', 'bye']:
            print("\n👋 Thanks for using the Falcon RAG Chatbot! Goodbye!")
            break

        if not user_input:
            print("⚠️ Please enter a question.")
            continue

        question_count += 1
        print(f"\n🔍 Processing question #{question_count}...")
        try:
            answer = ask_question(llm, db, user_input)
            print(f"\n💡 Answer #{question_count}:")
            print("-" * 40)
            print(answer)
            print("-" * 40)
        except Exception as e:
            print(f"❌ Error generating answer: {str(e)}")

# === MAIN ===
if __name__ == "__main__":
    pdf_folder_path = "/kaggle/input/documents/"  # update as needed

    print("🚀 Starting RAG Pipeline...")
    total_start_time = time.time()

    print("🔍 Loading PDFs...")
    docs = load_pdf_folder(pdf_folder_path)
    chunks = split_docs(docs)

    print("📚 Creating vectorstore...")
    db = create_vectorstore(chunks)

    print("🚀 Loading Falcon 7B model...")
    llm = load_falcon_model()

    print(f"✅ Setup completed in {time.time() - total_start_time:.2f} seconds")

    chatbot(llm, db)


🚀 Starting RAG Pipeline...
🔍 Loading PDFs...
📄 Loaded 153 documents in 9.46 seconds
✂️  Split into 2103 chunks in 0.05 seconds
📚 Creating vectorstore...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


🔍 Created vectorstore in 7.34 seconds
🚀 Loading Falcon 7B model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

🤖 Loaded Falcon 7B Instruct in 59.48 seconds
✅ Setup completed in 76.33 seconds

🤖 Falcon 7B RAG Chatbot
💡 Type 'quit' or 'exit' to end the session


...


❓ Your question:  Which factors affect brand competitiveness?


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



🔍 Processing question #3...
⏱️ Generated answer in 8.42 seconds

💡 Answer #3:
----------------------------------------
Brand competitiveness is affected by internal factors such as unique features, external image, and regional characteristics, as well as external factors such as market share, brand value creation capability, and competition.
----------------------------------------



❓ Your question:  quit



👋 Thanks for using the Falcon RAG Chatbot! Goodbye!


In [16]:
import os
import time
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# Step 1: Load and split PDFs
def load_pdf_folder(folder_path):
    start_time = time.time()
    documents = []
    for file in os.listdir(folder_path):
        if file.endswith(".pdf"):
            loader = PyPDFLoader(os.path.join(folder_path, file))
            documents.extend(loader.load())
    print(f"📄 Loaded {len(documents)} documents in {time.time() - start_time:.2f} seconds")
    return documents

# Step 2: Split into chunks
def split_docs(documents):
    start_time = time.time()
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = splitter.split_documents(documents)
    print(f"✂️  Split into {len(chunks)} chunks in {time.time() - start_time:.2f} seconds")
    return chunks

# Step 3: Embed documents
def create_vectorstore(docs):
    start_time = time.time()
    embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    db = Chroma.from_documents(docs, embedding)
    print(f"🔍 Created vectorstore in {time.time() - start_time:.2f} seconds")
    return db

# Step 4: Load LLaMA Model (GPU-enabled) - OPTIMIZED
def load_llama_model(model_id="meta-llama/Llama-2-7b-chat-hf"):
    start_time = time.time()
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    
    # Load LLaMA with flash attention disabled
    model = AutoModelForCausalLM.from_pretrained(
        model_id, 
        device_map="auto", 
        torch_dtype="auto",
        use_flash_attention_2=False  # Explicitly disable flash attention
    )
    
    # Optimized pipeline settings
    pipe = pipeline(
        "text-generation", 
        model=model, 
        tokenizer=tokenizer, 
        max_new_tokens=256,  # Reduced from 512 for faster responses
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1
    )
    
    llm = HuggingFacePipeline(pipeline=pipe)
    print(f"🤖 Loaded LLaMA model in {time.time() - start_time:.2f} seconds")
    return llm

# Step 5: Answer a question using RAG - OPTIMIZED
def ask_question(llm, db, question):
    start_time = time.time()
    
    # Get fewer, more relevant documents
    relevant_docs = db.similarity_search(question, k=2)  # Reduced from 3 to 2
    
    # Clean and truncate context
    context_parts = []
    for doc in relevant_docs:
        # Clean up the text (remove headers, footers, etc.)
        clean_text = doc.page_content.strip()
        # Limit each document to 300 characters
        if len(clean_text) > 300:
            clean_text = clean_text[:300] + "..."
        context_parts.append(clean_text)
    
    context = "\n\n".join(context_parts)
    
    # Optimized prompt for GPT-2
    prompt = f"""Based on the following context, provide a concise answer to the question.

Context: {context}

Question: {question}

Answer:"""
    
    response = llm.invoke(prompt)
    
    # Clean up the response to extract only the answer
    response_text = str(response)
    
    # Remove the prompt parts from the response
    if "Context:" in response_text:
        # Find where the actual answer starts
        answer_start = response_text.find("Answer:")
        if answer_start != -1:
            # Extract only the answer part
            answer = response_text[answer_start + 7:].strip()
        else:
            # If we can't find "Answer:", just return the last part
            answer = response_text.split("Question:")[-1].split("Answer:")[-1].strip()
    else:
        # If no "Context:" found, return the response as is
        answer = response_text
    
    # Clean up any remaining prompt artifacts
    answer = answer.replace("Answer the question using the context below.", "")
    answer = answer.replace("Context:", "").replace("Question:", "")
    answer = answer.strip()
    
    print(f"⏱️  Generated answer in {time.time() - start_time:.2f} seconds")
    return answer

# Step 6: Interactive Chatbot - OPTIMIZED
def chatbot(llm, db):
    print("\n" + "="*80)
    print("🤖 RAG CHATBOT - Ask questions about your documents!")
    print("💡 Type 'quit' or 'exit' to end the conversation")
    print("⚡ Optimized for faster responses (max 256 tokens)")
    print("="*80)
    
    question_count = 0
    
    while True:
        print("\n" + "-"*50)
        user_input = input("❓ Your question: ").strip()
        
        # Check for quit commands
        if user_input.lower() in ['quit', 'exit', 'q', 'bye']:
            print("\n👋 Thanks for using the RAG Chatbot! Goodbye!")
            break
        
        # Check for empty input
        if not user_input:
            print("⚠️  Please enter a question.")
            continue
        
        question_count += 1
        print(f"\n🔍 Processing question #{question_count}...")
        
        try:
            # Get answer
            answer = ask_question(llm, db, user_input)
            
            # Display answer
            print(f"\n💡 Answer #{question_count}:")
            print("-" * 40)
            print(answer)
            print("-" * 40)
            
        except Exception as e:
            print(f"❌ Error generating answer: {str(e)}")
            print("Please try again with a different question.")

# === MAIN EXECUTION ===
pdf_folder_path = "/kaggle/input/documents/"  # <- update this to where your 11 PDFs are

print("🚀 Starting RAG Pipeline...")
total_start_time = time.time()

print("🔍 Loading PDFs...")
docs = load_pdf_folder(pdf_folder_path)
chunks = split_docs(docs)

print("📚 Creating vectorstore...")
db = create_vectorstore(chunks)

print("🚀 Loading LLaMA model on GPU...")
llm = load_llama_model()

print(f"✅ Setup completed in {time.time() - total_start_time:.2f} seconds")

# Start the interactive chatbot
chatbot(llm, db)

print(f"\n🎉 Total session time: {time.time() - total_start_time:.2f} seconds") 

🚀 Starting RAG Pipeline...
🔍 Loading PDFs...
📄 Loaded 153 documents in 9.89 seconds
✂️  Split into 2103 chunks in 0.05 seconds
📚 Creating vectorstore...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


🔍 Created vectorstore in 8.04 seconds
🚀 Loading LLaMA model on GPU...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

🤖 Loaded LLaMA model in 7.04 seconds
✅ Setup completed in 25.02 seconds

🤖 RAG CHATBOT - Ask questions about your documents!
💡 Type 'quit' or 'exit' to end the conversation
⚡ Optimized for faster responses (max 256 tokens)

--------------------------------------------------


...


🔍 Processing question #2...
⏱️  Generated answer in 5.20 seconds

💡 Answer #2:
----------------------------------------
Luxury co-branding refers to the strategic partnership or collaboration between a luxury brand and a mass brand to create new products or services that combine the prestige and exclusivity of the luxury brand with the accessibility and affordability of the mass brand. This partnership allows both brands to leverage each other's strengths and resources to create mutually beneficial outcomes.
----------------------------------------

--------------------------------------------------


❓ Your question:  Which factors affect brand competitiveness?



🔍 Processing question #3...
⏱️  Generated answer in 1.82 seconds

💡 Answer #3:
----------------------------------------
According to scholarly studies, brand competitiveness is affected by various factors including internal features, external image, and regional characteristics.
----------------------------------------

--------------------------------------------------


❓ Your question:  quit



👋 Thanks for using the RAG Chatbot! Goodbye!

🎉 Total session time: 60.91 seconds
